# 동시에 여러개 이미지 다운로드


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from multiprocessing import Pool
import time
import os
import urllib.request
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### 폴더 생성

In [3]:
def create_folder(directory):
   
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print("error : Creating directory ... " + directory)

### 검색 키워드 호출

- csv 파일에 저장된 키워드를 추출해 검색 시작

In [4]:
keywords = []

key = pd.read_csv('./keyword.txt', encoding='utf-8', names=['keyword'])
[keywords.append(key['keyword'][x]) for x in range(len(key))]
keywords

['Aegean cat', 'American Bobtail cat', 'British Shorthair cat']

### 함수를 통한 이미지 다운로드 시작

- 앞과 비슷한 방법으로 실행

In [8]:
for i in range(len(keywords)):
    create_folder("./" +"0"+ str(i+1) + "."+keywords[i])

In [10]:
def image_download(keywords):
# 크롬 드라이브 호출
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    chromedriver = "./chromedriver"
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.implicitly_wait(3)

    # 검색
    print('검색 >> ', keywords)
    driver.get("https://www.google.co.kr/imghp?h1=ko")
    keyword = driver.find_element_by_xpath(
        '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input')
    keyword.send_keys(keywords)
    keyword.send_keys(Keys.RETURN)

    # 스크롤 내리기 -> 결과 더보기 버튼 클릭
    print("Scroll Down ..... ", keywords)
    elem = driver.find_element_by_tag_name('body')
    for i in range(60):
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.2)

    try:
        # //*[@id="islmp"]/div/div/div/div[2]/div[1]/div[2]/div[2]/input
        driver.find_element_by_xpath(
            '//*[@id="islmp"]/div/div/div/div[2]/div[1]/div[2]/div[2]/input').click()
        for i in range(60):
            elem.send_keys(Keys.PAGE_DOWN)
            time.sleep(0.2)
    except:
        pass

### 멀티 프로세스 적용

- processes=4로 설정하여 한번에 최대 4개 창을 동시에 활용할 수 있도록 적용


In [6]:
if __name__ == '__main__':
    pool = Pool(processes=3)
    pool.map(image_download, keywords)
